<a href="https://colab.research.google.com/github/sowmini4901/SceneSegmentation/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install focal-loss

In [ ]:
pip install tensorflow

In [ ]:
import pickle
import torch
import numpy as np
import sklearn
import os
from sklearn import preprocessing
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Suppress TF info
import tensorflow as tf
import matplotlib.pyplot as plt
from focal_loss import BinaryFocalLoss
import time


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
folder='/content/gdrive/MyDrive/movies/data'
files=os.listdir(folder)
total_files = len(files) 
shotsforMovie = np.zeros((total_files), dtype = np.uint16)

window = 7 # Window needs to be an int greater than 1 and odd!
first = int((window - 1)/2)

model1 = tf.keras.models.load_model('/content/gdrive/MyDrive/movies/data/softmax_1.h5', compile = True) 
# model2 = tf.keras.models.load_model('drive/MyDrive/Eluvio-MLChallenge/lasso/separable_lasso1.h5', compile=False)

In [ ]:
k=0
j=0
counter=0

In [ ]:
for i in range(total_files):
    
    filename = folder + '/' + files[i]
    f = open(filename, 'rb')
    data = pickle.load(f)
    f.close()
        
    feat1 = data['place']
    feat1 = feat1.data.numpy() #convert tensors into numpy arrays for sklearn
    feat1_size = feat1.shape[1]
    
    feat2 = data['cast']
    feat2 = feat2.data.numpy()
    feat2_size = feat2.shape[1]
    
    feat3 = data['action']
    feat3 = feat3.data.numpy()
    feat3_size = feat3.shape[1]
    
    feat4 = data['audio']
    feat4 = feat4.data.numpy()
    feat4_size = feat4.shape[1]
    
    x = np.hstack((feat1, feat2, feat3, feat4))
    y = data['scene_transition_boundary_ground_truth']
    N = x.shape[0] 
    x_fold = np.zeros((N - 1, 2*x.shape[1])) 
    start_s = time.time()

    for i in range(N - 1):
      x_fold[i,:] = np.hstack((x[i,:], x[i+1,:])) 
    
    if (j == 0):
      X = x_fold
      Y = y

    else:
      X = np.concatenate((X, x_fold), axis = 0)
      Y = np.concatenate((Y, y))
    
    j = j + 1
    k = k + 1

    print(X.shape)

    end_s = time.time()
    
    metrics2=model1.evaluate(X,y)
    metrics2
    
    #converting the predictions to 0 and 1 from false and true
    M = Y.shape[0]
  YTruth = np.zeros((M), dtype = np.uint8)
for i in range(M):
  if(Y[i] == True):
      YTruth[i] = 1
  elif(Y[i] == False):
      YTruth[i] = 0


    a = data['scene_transition_boundary_ground_truth']
    a = a.data.numpy()
